# Streaming data from micro-manager to Napari: PSF Viewer

This notebook shows how to acquire data using `micromanager`, then use `pycro-manager` to stream it to `napari`.  
Buttons to start and stop data acquisition are added to `napari` using the `magic-gui` package.  
In this example, the data displayed in `napari` resliced to get a live PSF viewer. However, reslicing is only a small example for the data analysis possible using `napari`.

In [1]:
# only execute first time to install all required packages
#!pip install pycromanager queue napari pyqt5 magicgui

In [2]:
import time
import numpy as np
import queue

import napari
from napari.qt import thread_worker

from magicgui import magicgui

from pycromanager import Dataset, Acquisition

# open napari in an extra window
%gui qt

# define constants

In [3]:
# data acquired on microscope or simulated?
simulate = True
# image size
size = [2048, 2048]
# clip image to central part. Speeds up display as data size is reduced
clip =[500, 500]
# um / px for scaling in napari
size_um = [1, 1]
# start in um, end in um, number of slices, active slice
z_range = [0, 512, 100, 0]
# sleep time to keep software responsive
sleep_time = 0.05
# contrast limits for display
clim = [0, 200]
# color map for display
cmap = 'plasma'

# initialize global variables
acq_running = False
img_queue = queue.Queue()
data = np.random.rand(z_range[2], clip[0], clip[1]) * clim[1]

# create dummy image and and put into stack
adds dummy image of constant brightness  
keeps track of z position  
adds image and z position to queue  
use for testing purposes without microscope  
stack of increasing brightness helps to identify glitches

In [4]:
def simulate_image(b, size = [128,128]):
    """ fnc to simulate an image of constant brightness.
        Keeps track of z-position in stacks, concatenates
        image data and z-position and adds to queue.
        Inputs: int b: brightness
                np.array size: # of px in image in xy.
        Global variables: img_queue to write image and z position to
                z_range to keep track of z position
        """
    global img_queue
    global z_range
    image = np.ones(size) * b
    image_clipped = image[(size[0]-clip[0])//2:(size[0]+clip[0])//2,
                          (size[1]-clip[1])//2:(size[1]+clip[1])//2]
    img_queue.put([z_range[3], np.ravel(image_clipped)])
    z_range[3]= (z_range[3]+1) % z_range[2]

In [5]:
def simulate_data(ii, z_range):
    """ fnc to create images with constant, but increasing brightness.
        Inputs: int ii: counter to increase brightness
                int z_range: number of slices in stack"""
    for zz in range(z_range):
        brightness = (ii+1) * (zz+1) / ((z_range+1)) * clim[1]
        simulate_image(brightness, size)

# image process function and pycromanager acquisition
adds acquired image and z position to queue,  
keeps track of z position  
built pycromanager acquisition events  
acquire data and send to image_process_fn  

In [6]:
def grab_image(image, metadata):
    """ image_process_fnc to grab image from uManager.
        Keeps track of z-position in stacks, concatenates
        image data and z-position and adds to queue.
        Inputs: int b: brightness
                np.array size: # of px in image in xy.
        Global variables: img_queue to write image and z position to
                z_range to keep track of z position
        """
    global img_queue
    global z_range

    image_clipped = image[(size[0]-clip[0])//2:(size[0]+clip[0])//2,
                      (size[1]-clip[1])//2:(size[1]+clip[1])//2]
    img_queue.put([z_range[3], np.ravel(image_clipped)])
    z_range[3]= (z_range[3]+1) % z_range[2]
   
    return image, metadata

In [7]:
def acquire_data(z_range):
    """ micro-manager data acquisition. Creates acquisition events for z-stack.
        This example: use custom events, not multi_d_acquisition because the 
        z-stage is not run from micro-manager but controlled via external DAQ."""
    with Acquisition(directory=None, name=None, 
                     show_display=True, 
                     image_process_fn = grab_image) as acq:
        events = []
        for index, z_um in enumerate(np.linspace(z_range[0], z_range[1], z_range[2])):
            evt = {"axes": {"z_ext": index}, "z_ext": z_um}
            events.append(evt)
        acq.acquire(events)

# napari update display

In [8]:
def display_napari(pos_img):
    """ Unpacks z position and reshapes image from pos_img. Writes image into correct 
        slice of data, and updates napari display.
        Called by worker thread yielding elements from queue.
        Needs to be in code before worker thread connecting to it.
        Inputs: array pos_img: queue element containing z position and raveled image data.
        Global variables: np.array data: contains image stack
            img_queue: needed only to send task_done() signal.
    """
    global data
    global img_queue
    if pos_img is None:
        return
    # read image and z position
    image = np.reshape(pos_img[1:],(clip[0], clip[1]))
    z_pos = pos_img[0]

    # write image into correct slice of data and update display
    data[z_pos] = np.squeeze(image)
    layer = viewer.layers[0]
    layer.data = data

    img_queue.task_done()

# worker threads appending data to queue and reading from queue

In [9]:
@thread_worker
def append_img(img_queue):
    """ Worker thread that adds images to a list.
        Calls either micro-manager data acquisition or functions for simulating data.
        Inputs: img_queue """
    # start microscope data acquisition
    if not simulate:
        while acq_running:
            acquire_data(z_range)
            time.sleep(sleep_time)

    # run with simulated data
    else:
        ii = 0
        while acq_running:
            simulate_data(ii, z_range[2])
            ii = ii + 1
            time.sleep(sleep_time)

In [10]:
@thread_worker(connect={'yielded': display_napari})
def yield_img(img_queue):
    """ Worker thread that checks whether there are elements in the 
        queue, reads them out.
        Connected to display_napari function to update display """
    global acq_running
    
    while acq_running:
        time.sleep(sleep_time)
        # get elements from queue while there is more than one element
        # playing it safe: I'm always leaving one element in the queue
        while img_queue.qsize() > 1:
            yield img_queue.get(block = False)

    # read out last remaining elements after end of acquisition
    while img_queue.qsize() > 0:
        yield img_queue.get(block = False)
    print("acquisition done")

# define functions to start and stop acquisition
connect to gui buttons using magic_gui

In [11]:
@magicgui(call_button="Start")
def start_acq():
    print("starting threads...")
    global acq_running
        
    if not(acq_running):
        worker1 = append_img(img_queue)
        worker2 = yield_img(img_queue)
        acq_running = True
        worker1.start()
        #worker2.start() # doesn't need to be started bc yield is connected
    else:
        print("acquisition already running!")
    
    
@magicgui(call_button = "Stop")
def stop_acq():
    print("stopping threads")
    # set global acq_running to False to stop other workers
    global acq_running
    acq_running = False

# "Main" function: start napari and worker threads

In [12]:
# check if viewer is already open
# if yes: close and reopen
try:
    if viewer:
        viewer.close()
except:
    print("viewer already closed or never opened")
viewer = napari.Viewer(ndisplay=2)

# initialize napari viewer with stack view and random data, reslice view
layer = viewer.add_image(data, 
                        name = 'uManager',
                        colormap = cmap,
                        interpolation = 'nearest',
                        blending = 'additive',
                        rendering = 'attenuated_mip',
                        scale = [z_range[1]/z_range[2], size_um[1], size_um[0]],
                        contrast_limits = clim )
viewer.dims._roll()

# define start stop buttons and add to napari gui
gui_start = start_acq.Gui()
gui_stop = stop_acq.Gui()
viewer.window.add_dock_widget(gui_start)
viewer.window.add_dock_widget(gui_stop)

viewer already closed or never opened


starting threads...
stopping threads
acquisition done
starting threads...
stopping threads
acquisition done
starting threads...
stopping threads
acquisition done
